# IMPORT PYTHON MODULES

In [2]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

# IMPORT AND VALIDATE DATA 

In [3]:
warnings.filterwarnings("ignore")

#import data
data = r"C:\Users\katdovzh\Desktop\projects\campaigns_analysis\dataset\marketing_data.csv"
cp = pd.read_csv(data)

cp.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,...,6,1,0,0,0,0,0,1,0,Spain
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,...,7,5,0,0,0,0,1,1,0,Canada
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,...,5,2,0,0,0,0,0,0,0,USA
3,1386,1967,Graduation,Together,32474.0,1,1,2014-05-11,0,10,...,2,7,0,0,0,0,0,0,0,Australia
4,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,6,...,2,7,1,0,0,0,0,1,0,Spain


In [ ]:
cp.info()

# CLEANING DATA

In [ ]:
# Remove users without medium path info
data_filtered = cp[cp['medium_path'].notnull()]
data_filtered['first_touchpoint'] = pd.to_datetime(data_filtered['first_touchpoint'])
data_filtered['conversion_timestamp'] = pd.to_datetime(data_filtered['conversion_timestamp'])

In [ ]:
data_filtered.head()

# PREPROCESS DATA

In [ ]:
converted_users = data_filtered[data_filtered['conversion_flag'] == 1]

expanded_paths_conv_user = converted_users['medium_path'].str.split(',', expand=True)

not_converted_users = data_filtered[data_filtered['conversion_flag'] == 0]

expanded_paths_non_conv_user = not_converted_users['medium_path'].str.split(',', expand=True)



In [ ]:
# Define the maximum number of steps
# Why 13? 99% of users have max 13 steps, see details 'data_analysis'
MAX_STEPS = 13


# Function to split and expand medium paths
def expand_medium_path(path):
    if pd.notnull(path):
        steps = path.split(',')
        return pd.Series(steps)
    else:
        return pd.Series([None])

# Apply the function to the medium_path column
expanded_paths = data_filtered['medium_path'].apply(expand_medium_path)

expanded_paths_first_5 = expanded_paths.iloc[:, :MAX_STEPS]

final_df = pd.concat([data_filtered, expanded_paths_first_5], axis=1)


In [ ]:
# Create a list of column names
column_names = ['user_pseudo_id', 'medium_path', 'conversion_flag', 'first_touchpoint', 'conversion_timestamp']
for i in range(1, MAX_STEPS + 1):
    column_names.append(f'step{i}')

# Assign the column names to the DataFrame
final_df.columns = column_names

In [ ]:
final_df.head(15)

# STORE FINAL DATASET

In [ ]:
final_df.to_csv('final_dataset.csv')